In [47]:
import pandas as pd
import numpy as np
import gensim

In [67]:
data = pd.read_csv('uci-news-aggregator.csv')
data=data.sample(frac=1)

In [68]:
data

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
170231,170567,"Health Highlights: April 29, 2014",http://www.philly.com/philly/health/topics/Hea...,Philly.com,m,dt6AZxnL6V2CLfMWw8lTk0ziYi6eM,www.philly.com,1398802528739
254326,254772,"Model-turned-pin-up photographer Bunny Yeager,...",http://www.montrealgazette.com/entertainment/M...,Montreal Gazette,e,dP7VIVTpjCL9FvMH5t6H3gvPKshgM,www.montrealgazette.com,1401174383387
141682,142018,Remarks by the First Lady at Discussion with H...,http://www.newsroomamerica.com/story/417893.html,Newsroom America,e,d23xRqJsAx3PWJMC7R_qov83sshqM,www.newsroomamerica.com,1398004160592
106068,106265,Will Amazon Release a Smartphone With a 3D Dis...,http://www.pastemagazine.com/articles/2014/04/...,Paste Magazine,t,dyha9eq-DLqP2zM5ynyDTl35a7JfM,www.pastemagazine.com,1397351121621
34773,34774,"Niluksi Koswanage and Matt Siegel, Reuters",http://www.thebarrieexaminer.com/2014/03/23/fr...,The Barrie Examiner,b,dx-dgpjwhPTym3MiI6I2HolKsFQMM,www.thebarrieexaminer.com,1395616669567
368233,368693,Motorola Moto G vs. Apple iPhone 5C: In-depth ...,http://airherald.com/motorola-moto-g-vs-apple-...,Air Herald,t,dSrhp8EaF5XWAOMDdviPUjyeL_fKM,airherald.com,1404830572555
314339,314799,Posted by Muhammad Iqbal,http://www.brecorder.com/markets/equity/europe...,Business Recorder,b,d02UFXK26SEszSM8Q1X2nUHj2pcOM,www.brecorder.com,1403700923517
183669,184005,'Fountain of Youth' Found in Blood of Young Mi...,http://au.ibtimes.com/articles/551263/20140506...,International Business Times AU,m,dS6w6bx-tFJ96cMYBawErLk9l-UPM,au.ibtimes.com,1399387742643
358866,359326,Thousands march against Ebola in Sierra Leone,http://www.sierraexpressmedia.com/archives/68719,Sierra Express Media,m,dvOcpD5z3AbEILM2mxYrnqHY6CbZM,www.sierraexpressmedia.com,1404537850913
166906,167242,AOL traces mystery spam flood to security brea...,http://www.pcworld.com/article/2148523/aol-tra...,PCWorld \(blog\),t,d9-6h2PKv6IVMnMk8dphTDedl28QM,www.pcworld.com,1398781129409


In [27]:
# Load Google's pre-trained Word2Vec model.
w2vmodel = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [28]:
#w2vmodel.wv.most_similar(positive='compute')

C:\Users\Rohit\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('computation', 0.6541281342506409),
 ('computations', 0.6274853944778442),
 ('calculate', 0.6102968454360962),
 ('computing', 0.5723111033439636),
 ('compute_cycles', 0.5714893341064453),
 ('compute_intensive', 0.5660620927810669),
 ('Compute', 0.5589189529418945),
 ('parallelize', 0.5471308827400208),
 ('computational', 0.5439306497573853),
 ('computes', 0.5419020652770996)]

In [89]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        #if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
        result.append(lemmatize_stemming(token))
    return result

In [90]:
data.shape[0]

422419

In [109]:
featX = np.zeros((data.shape[0], 300), dtype=float)
featY = np.zeros((data.shape[0]), dtype = str)
idx = 0
for index,row in data.iterrows():
    title = row[1]
    title = preprocess(title)
    vec = np.zeros(300,dtype=float)
    for w in title:
        # Get the embedding for word w
        if w not in w2vmodel: continue
        vec = w2vmodel[w]
        featX[idx] += vec
    # Average the feature vector by the length of the title
    if len(title) != 0: featX[idx] = featX[idx]/float(len(title))
        
    # Create the label for these feature vectors
    featY[idx] = row[4]
    if featY[idx] == 'b': featY[idx] = 0
    if featY[idx] == 'e': featY[idx] = 1
    if featY[idx] == 'm': featY[idx] = 2
    if featY[idx] == 't': featY[idx] = 3
    idx += 1

In [110]:
len(featX)

422419

In [115]:
X_train = featX[:350000]
y_train = featY[:350000]
X_vaildate = featX[350001:]
y_validate = featY[350001:]

In [116]:
featY[50000:50030]

array(['0', '2', '1', '1', '3', '2', '1', '1', '3', '1', '2', '0', '1',
       '1', '1', '2', '2', '1', '3', '3', '2', '0', '1', '1', '0', '0',
       '3', '3', '0', '3'], dtype='<U1')

In [ ]:
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train) 
svm_predictions = svm_model_linear.predict(X_validate)

In [107]:
# model accuracy for X_test   
accuracy = svm_model_linear.score(X_test, y_test) 